In [2]:
!git clone https://github.com/CodingYodha/Legal_TExt_summarizer.git

fatal: destination path 'Legal_TExt_summarizer' already exists and is not an empty directory.


In [4]:

from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [ ]:
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch

In [6]:
dataset = load_dataset("csv" ,
                       data_files = {"train":"/kaggle/input/erwtwret/legal_train.csv" ,
                                     "validation":"/kaggle/input/erwtwret/legal_val.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn",
                                                    use_cache = False) #disables caching for checkpointing)
model.gradient_checkpointing_enable()#reduces memory by 60-70%
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

In [8]:
def preprocess_function(examples):
    inputs =  examples["text"]
    targets =  examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=1024,
        truncation=True,
        # padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=256,
            truncation = True,
            # padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/7082 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1771 [00:00<?, ? examples/s]

barts max input size is 1024 tokens

datacollator

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer ,
                                       model = model,
                                      padding="longest")

purpose of data collator - dynamic padding the batches during training for efficient memeory usage

Training arguements

In [10]:

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",

    learning_rate = 3e-5,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay = 0.01,
    save_total_limit = 3,
    fp16=True, #use GPU mixed precision
    logging_steps=50,
    report_to = "none",
    gradient_accumulation_steps=2,

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


fp16=True: Speeds up training on GPUs with Tensor Cores (NVIDIA).

per_device_batch_size: Adjust based on GPU memory (start with 2-8).

trainer setup

In [11]:
# import torch
# import os

# # Clear cached memory
# torch.cuda.empty_cache()

# # Reduce fragmentation
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [12]:
# PYTORCH_CUDA_ALLOC_CONF=expandable_segments=True

In [13]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset = tokenized_dataset["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

<ipython-input-13-cf7d7f3da60b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
import torch
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

GPU available: True
GPU device: Tesla P100-PCIE-16GB


In [15]:
# Move model to GPU
# model = model.to("cuda")

# # Verify model is on GPU
# print(next(model.parameters()).device)  # Should output "cuda:0"

start training

save the model

In [16]:
small_dataset = tokenized_dataset["train"].shuffle().select(range(7000))

# Force reinitialization of Trainer internals
trainer.train_dataset = small_dataset
trainer._train_dataloader = None  # Reset dataloader


# Verify GPU usage
print(f"Model device: {next(model.parameters()).device}")
print(f"CUDA memory allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# Start training
trainer.train()

Model device: cuda:0
CUDA memory allocated: 3.25 GB


Epoch,Training Loss,Validation Loss
1,2.987600,2.905222
2,2.328600,2.832303
3,1.812400,2.882676


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=5250, training_loss=2.469088685535249, metrics={'train_runtime': 2150.7795, 'train_samples_per_second': 9.764, 'train_steps_per_second': 2.441, 'total_flos': 2582037411397632.0, 'train_loss': 2.469088685535249, 'epoch': 3.0})

In [17]:
import shutil
from zipfile import ZipFile

# 1. Save model & tokenizer (if not already saved)
model.save_pretrained("legal_bart_summarizer")
tokenizer.save_pretrained("legal_bart_summarizer")

# 2. Create zip archive
shutil.make_archive(
    base_name="/kaggle/working/legal_bart_summarizer",  # Save in Kaggle's working directory
    format="zip",
    root_dir="legal_bart_summarizer",
)

# 3. Verify zip contents
with ZipFile("/kaggle/working/legal_bart_summarizer.zip", "r") as zip:
    print("Zip file contains:", zip.namelist())

print("\n✅ Download ready! Click the 📁 icon in Kaggle's sidebar →")
print("→ Output → legal_bart_summarizer.zip")

Zip file contains: ['config.json', 'tokenizer_config.json', 'model.safetensors', 'generation_config.json', 'vocab.json', 'merges.txt', 'special_tokens_map.json']

✅ Download ready! Click the 📁 icon in Kaggle's sidebar →
→ Output → legal_bart_summarizer.zip


In [ ]:
# small_dataset = tokenized_dataset["train"].shuffle().select(range(2000))
# trainer.train_dataset = small_dataset
# trainer.train()

In [ ]:
model.save_pretrained("legal_bart_summarizer")
tokenizer.save_pretrained("legal_bart_summarizer")

inference

In [18]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load model and tokenizer from the saved directory
model = BartForConditionalGeneration.from_pretrained("legal_bart_summarizer")
tokenizer = BartTokenizer.from_pretrained("legal_bart_summarizer")

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
input_text = """HELD: The Court was concerned with the narrow question whether the six law reports aforementioned being published by the 1st respondent were 
newspapers within the meaning of the Act and whether the employees engaged in their 
production or distribution were entitled to the benefit of the orders made by the Central Government on the basis of the Palekar Award.
"""
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024)
summary_ids = model.generate(inputs["input_ids"], max_length=256)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1527: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


The question was whether the 6 law reports were newspapers within the meaning of the Act.


In [ ]:
summary

In [ ]:
model.push_to_hub("Shiva7706/legal-bart-summarizer")
tokenizer.push_to_hub("Shiva7706/legal-bart-summarizer")

In [ ]:
from IPython.display import FileLink, display

# Path to your zip file
zip_path = "/kaggle/working/legal_bart_summarizer.zip"

# Create a downloadable link
display(FileLink(zip_path, result_html_prefix="⬇️ Click to download: "))

# Verify file exists
!ls -lh /kaggle/working/legal_bart_summarizer.zip